In [2]:
import VESIcal as v
import pandas as pd
import numpy as np
import os
import Thermobar as pt
import multiprocessing as mp
import sys
import io
from tqdm import tqdm
import gc

# %pip install Thermobar

In [4]:
out=pt.import_excel("MI_compilation_XH2O.xlsx", sheet_name="XH2O_forThermobar")

# This subdivdes outputs into a dataframe for all inputs (my_input), ols, and liqs
my_input=out['my_input']
myOls=out['Ols']
myLiquids1=out['Liqs']

## Lets check the outputs have loaded right
display(myOls.head())
display(myLiquids1.head())

,SiO2_Ol,TiO2_Ol,Al2O3_Ol,FeOt_Ol,MnO_Ol,MgO_Ol,CaO_Ol,Na2O_Ol,K2O_Ol,Cr2O3_Ol,NiO_Ol,Sample_ID_Ol
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4


,SiO2_Liq,TiO2_Liq,Al2O3_Liq,FeOt_Liq,MnO_Liq,MgO_Liq,CaO_Liq,Na2O_Liq,K2O_Liq,Cr2O3_Liq,P2O5_Liq,H2O_Liq,Fe3Fet_Liq,NiO_Liq,CoO_Liq,CO2_Liq,Sample_ID_Liq
0,49.5551,1.999709,13.45010,13.569085,0.233964,6.163391,11.14230,2.46200,0.247106,0.021049,0.137950,0.06790,0.0,0.000000,0.0,0.002328,Bali_2018_Holohraun_1
1,48.8914,1.941204,13.47300,14.000915,0.236328,6.456140,11.45715,2.29605,0.205395,0.014379,0.226211,0.07720,0.0,0.017466,0.0,0.019493,Bali_2018_Holohraun_2
2,49.3044,1.824232,13.68435,13.305956,0.250050,6.560064,11.39425,2.51540,0.213113,0.013414,0.183380,0.16405,0.0,0.024897,0.0,0.025766,Bali_2018_Holohraun_3
3,49.9440,1.834000,13.32000,12.576778,0.207592,6.395452,11.22520,2.56000,0.278156,0.013640,0.158312,0.17600,0.0,0.000000,0.0,0.026488,Bali_2018_Holohraun_4
4,50.4000,1.920000,14.01000,11.289602,0.220000,6.920000,11.69000,2.38000,0.240000,0.001260,0.240000,0.45600,0.0,0.028665,0.0,0.027455,Bali_2018_Holohraun_5


In [5]:
T_put_2008=pt.calculate_liq_only_temp(liq_comps=myLiquids1,equationT="T_Put2008_eq22_BeattDMg", 
                           P=my_input['SaturationP_bars_VESIcal_TCaO'].values/1000,H2O_Liq=my_input['H2O_Liq'])
T_put_2008.head()

0    1435.587015
1    1444.856599
2    1444.932337
3    1439.731210
4    1438.242044
dtype: float64

In [6]:
df_Tput2008=pt.convert_to_vesical(liq_comps=myLiquids1, 
                              T1=T_put_2008,unit='Kelvin', Fe3Fet_Liq=0.15)
df_Tput2008.head()

,SiO2,TiO2,Al2O3,MnO,MgO,CaO,Na2O,K2O,Cr2O3,P2O5,H2O,NiO,CoO,CO2,Sample_ID,Temp,FeO,Fe2O3
0,49.5551,1.999709,13.45010,0.233964,6.163391,11.14230,2.46200,0.247106,0.021049,0.137950,0.06790,0.000000,0.0,0.002328,Bali_2018_Holohraun_1,1162.437015,11.533722,2.261514
1,48.8914,1.941204,13.47300,0.236328,6.456140,11.45715,2.29605,0.205395,0.014379,0.226211,0.07720,0.017466,0.0,0.019493,Bali_2018_Holohraun_2,1171.706599,11.900778,2.333486
2,49.3044,1.824232,13.68435,0.250050,6.560064,11.39425,2.51540,0.213113,0.013414,0.183380,0.16405,0.024897,0.0,0.025766,Bali_2018_Holohraun_3,1171.782337,11.310063,2.217659
3,49.9440,1.834000,13.32000,0.207592,6.395452,11.22520,2.56000,0.278156,0.013640,0.158312,0.17600,0.000000,0.0,0.026488,Bali_2018_Holohraun_4,1166.581210,10.690261,2.096129
4,50.4000,1.920000,14.01000,0.220000,6.920000,11.69000,2.38000,0.240000,0.001260,0.240000,0.45600,0.028665,0.0,0.027455,Bali_2018_Holohraun_5,1165.092044,9.596162,1.881600


In [ ]:
## Convert to VESIcal batchfile
VESIcal_out = v.BatchFile(filename=None, dataframe=df_Tput2008, label='Sample_ID')

In [39]:
#### Now we will run the calcs, using multiprocessing #########

##### This sets your input data and the chunk size. Input has to be a dataframe. 

df = VESIcal_out.data [0:85]

num_cores = mp.cpu_count() ## leave this as is if you don't know the number you want to use. 

chunk_size = 32 ## if you don't know what to do use chunk_size=num_cores. Don't use less. 
####### BUT if your chunks are too big, and the memory gets saturated, it will freeze the whole thing. so be careful. 
print("Number of processors: ", num_cores)

Number of processors:  8


In [40]:
########## Multiprocessing functions ###########
#################### These are the worker functions, don't touch unless you know what you're doing ############

def worker_function(queue, i, row):
    sys.stdout = io.StringIO()
    sub = pd.DataFrame([row])
    sub.index = [i]  # Preserve the original index
    sub_batchfile = v.BatchFile(filename=None, dataframe=sub, label=sub.index)
    
    try:
        sub_satP = sub_batchfile.calculate_saturation_pressure(temperature="Temp")
    except Exception as e:
        print(f"An error occurred: {e}")
        sub_satP = sub_batchfile.data.copy()
        sub_satP['SaturationP_bars_VESIcal'] = np.nan
        sub_satP['XH2O_fl_VESIcal'] = np.nan
        sub_satP['XCO2_fl_VESIcal'] = np.nan
        sub_satP['FluidMass_grams_VESIcal'] = np.nan
        sub_satP['FluidSystem_wt_VESIcal'] = np.nan
        sub_satP['Model'] = 'MagmaSat'
        sub_satP['Warnings'] = 'Could not converge'
    
    queue.put({'i': i, **sub_satP.to_dict(orient='records')[0]})
    gc.collect()


def process_chunk(chunk, num_cores):
    manager = mp.Manager()
    queue = manager.Queue()
    
    rows = chunk.to_dict('index')
    original_indices = list(rows.keys())  # Store the original order of indices
    pool = mp.Pool(num_cores)
    
    args = [(queue, i, row) for i, row in rows.items()]
    
    results = [pool.apply_async(worker_function, arg) for arg in args]
    
    pool.close()
    
    df_step = pd.DataFrame()
    
    processed = 0
    total = len(rows)
    
    with tqdm(total=total) as pbar:
        while processed < total:
            result = queue.get()
            processed += 1
            pbar.update(1)
            
            i = result.pop('i')
            df_step = pd.concat([df_step, pd.DataFrame(result, index=[i])])
            
            # Clear memory incrementally
            gc.collect()
    
    pool.join()
    
    # Sort the DataFrame to match the input order
    df_step = df_step.loc[original_indices]
    
    return df_step

########### This runs the calculations ###############
#### You should not need to edit this ################

print("Number of processors: ", num_cores)

if __name__ == '__main__':
    start_time = time.time()  # Start the timer

    num_chunks = (len(mp_df_in) + chunk_size - 1) // chunk_size  # Calculate the number of chunks
    
    final_df = pd.DataFrame()
    original_indices = []  # Store original indices
    
    for chunk_index in range(num_chunks):
        start_index = chunk_index * chunk_size
        end_index = min((chunk_index + 1) * chunk_size, len(mp_df_in))
        chunk = mp_df_in.iloc[start_index:end_index]
        
        # Store original indices of the chunk
        original_indices.extend(chunk.index)
        
        chunk_result = process_chunk(chunk, num_cores)
        
        final_df = pd.concat([final_df, chunk_result])
    
    # Sort the final_df based on the original indices
    final_df = final_df.reindex(original_indices)
    
    # Display the sorted final_df
    display(final_df.head())
    
    end_time = time.time()  # End the timer
    elapsed_time = end_time - start_time  # Calculate the elapsed time
    print(f"Total run time: {elapsed_time:.2f} s")

Number of processors:  8


100%|██████████| 21/21 [00:26<00:00,  1.24s/it]


,SiO2,TiO2,Al2O3,MnO,MgO,CaO,Na2O,K2O,Cr2O3,P2O5,...,FeO,Fe2O3,Temp,SaturationP_bars_VESIcal,XH2O_fl_VESIcal,XCO2_fl_VESIcal,FluidMass_grams_VESIcal,FluidSystem_wt_VESIcal,Model,Warnings
Bali_2018_Holohraun_1,49.5551,1.999709,13.45010,0.233964,6.163391,11.14230,2.46200,0.247106,0.021049,0.137950,...,11.533722,2.261514,1162.437015,30,0.031593,0.968407,0.000556,0.000556,MagmaSat,
Bali_2018_Holohraun_2,48.8914,1.941204,13.47300,0.236328,6.456140,11.45715,2.29605,0.205395,0.014379,0.226211,...,11.900778,2.333486,1171.706599,320,0.004356,0.995644,0.000035,0.000035,MagmaSat,
Bali_2018_Holohraun_3,49.3044,1.824232,13.68435,0.250050,6.560064,11.39425,2.51540,0.213113,0.013414,0.183380,...,11.310063,2.217659,1171.782337,420,0.014479,0.985521,0.000476,0.000476,MagmaSat,
Bali_2018_Holohraun_4,49.9440,1.834000,13.32000,0.207592,6.395452,11.22520,2.56000,0.278156,0.013640,0.158312,...,10.690261,2.096129,1166.581210,440,0.015395,0.984605,0.000100,0.000100,MagmaSat,
Bali_2018_Holohraun_5,50.4000,1.920000,14.01000,0.220000,6.920000,11.69000,2.38000,0.240000,0.001260,0.240000,...,9.596162,1.881600,1165.092044,480,0.086968,0.913032,0.000423,0.000423,MagmaSat,


Total run time: 90.36 s


In [ ]:
#################################### END OF NEW VERSION ################################